## Summary of data
## Properties from San Jose, CA
## 6099 condos
## 92 different neighborhoods

In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.path as path

from string import punctuation

In [2]:
def make_histogram(data, bins):
    fig, ax = plt.subplots()

    # histogram our data with numpy
    n, bins = np.histogram(data, bins)

    # get the corners of the rectangles for the histogram
    left = np.array(bins[:-1])
    right = np.array(bins[1:])
    bottom = np.zeros(len(left))
    top = bottom + n


    # we need a (numrects x numsides x 2) numpy array for the path helper
    # function to build a compound path
    XY = np.array([[left, left, right, right], [bottom, top, top, bottom]]).T

    # get the Path object
    barpath = path.Path.make_compound_path_from_polys(XY)

    # make a patch out of it
    patch = patches.PathPatch(barpath)
    ax.add_patch(patch)

    # update the view limits
    ax.set_xlim(left[0], right[-1])
    ax.set_ylim(bottom.min(), top.max())

    plt.show()

In [43]:
# convert string percentages to numerical
def convert_percentage(df):
    for col in df.columns:
        c = col.split(" ")
        if c[-1] == "MoM" or c[-1] == "YoY": df[col] = [float(str(x).strip("%")) for x in df[col]]
    
    df['Average Sale To List'] = [float(str(x).strip("%")) for x in df['Average Sale To List']]

In [4]:
### read in data
def read_data(filename, property_type):
    df = pd.read_csv(filename)
    df.columns = [x.strip(" ") for x in df.columns]
    df['property_type'] = property_type
    df.head()
    return (df)

In [5]:
### add columns for city, state, and neighborhood
def parse_region(df):
    df['location'] = [x.split(",") for x in df.Region]
    df['city'] = [x[0].strip(",").lower() for x in df.location]
    df['state'] = [x[1].split(" ")[1].lower().strip() for x in df.location]
    df['neighborhood'] = [x[1].split(" ")[3].strip().lower() for x in df.location]
    df = df[df.city == 'san jose']
    df = df.drop('location', 1)
    df.head()

In [6]:
### convert Median Sale Price to a float
def convert_med_sale_pr(df):
    df[u'Median Sale Price'] = [x.strip("$").replace(",", "") for x in df["Median Sale Price"]]
    df[u'Median Sale Price'] = [int(x) if x[len(x)-1] != "K" else int(x.strip("K"))*1000 
                                for x in df["Median Sale Price"]]

In [44]:
### given the filename and property type return
### a dataframe with string fields converted to
### numerical and regions parsed out
def clean_data(filename, property_type):
    df = read_data(filename, property_type)
    convert_med_sale_pr(df)
    parse_region(df)
    convert_percentage(df)
    return (df)

In [37]:
# merge several dataframes together
# files = list of tuples, 
# each tuple should contain filename first then property type

# How to deal with NA's? fill with 0 for now ...
def merge_data(files):
    frames = []
    for f in files:
        frames.append(clean_data(f[0], f[1]))
    return (pd.concat(frames).fillna(0))

In [51]:
### convert neighborhood to a numerical value
def convert_neighborhood(df):
    neighborhoods = df.neighborhood.unique()
    df['neighborhood_num'] = [np.nonzero(df.neighborhood == x)[0][0] for x in df.neighborhood]

In [52]:
files = [('sanjose_condos.csv', 'condo'), ('sanjose_townhouse2.csv', 'townhouse'),
        ('sanjose_multiunit.csv', 'multiunit'), ("sanjose_singlefamily.csv", "singlefamily")]
total = merge_data(files)
convert_neighborhood(total)

In [40]:
len(total.index)

15419

In [41]:
neighborhoods = np.unique(total['neighborhood'])
print (neighborhoods, len(neighborhoods)) # 92 neighborhoods

(array(['alexander', 'alma-almaden', 'almaden', 'anderson', 'area',
       'atlanta', 'barbera-stokes', 'berryessa', 'blackford',
       'bonita-24th', 'branham-jarvis', 'branham-kirk', 'brigadoon',
       'broadway-palmhaven', 'brooktree', 'bucknall', 'buena', 'calabazas',
       'cambrian', 'cambrian-pioneer', 'carlton', 'carson',
       'cedarville-giannotta', 'central', 'chaboya', 'cherrywood',
       'citaldi', 'clayton', 'commodore', 'crossgate', 'cypress-jurdo',
       'de', 'deer', 'del', 'dovehill', 'downtown', 'east', 'eden',
       'erikson', 'estates', 'evergreen', 'everydale-neimen', 'flickinger',
       'fowler', 'gilchrist', 'great', 'greylands', 'hammer', 'hayes',
       'heritage', 'hidden', 'hillsdale', 'hillview', 'holy', 'joaquin',
       'kenwood', 'king', 'kooser', 'lanai-cunningham', 'loma', 'lone',
       'los', 'lynbrook', 'lynhaven', 'mayfair', 'mckay-ringwood',
       'meadow', 'meadows', 'millic-phelps', 'miner', 'mirassou',
       'morrill', 'mount', 'muril

In [42]:
total.describe() # descriptive statistics

,Median Sale Price,Median Sale Price MoM,Median Sale Price YoY,Homes Sold,Homes Sold MoM,Homes Sold YoY,New Listings,New Listings MoM,New Listings YoY,Inventory,Inventory MoM,Inventory YoY,Days on Market,Days on Market MoM,Days on Market YoY,Average Sale To List,Average Sale To List MoM,Average Sale To List YoY
count,1.541900e+04,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000,15419.000000
mean,6.780970e+05,1.441598,13.424645,108.108113,7.759433,18.569492,135.838641,6.041780,12.451365,317.767949,11.687794,10.796414,26.702056,-0.146702,-3.628964,103.110020,0.039043,0.771289
std,2.963520e+05,10.651508,21.476353,646.383152,47.591472,92.052994,837.190488,45.285962,80.893613,2164.869063,65.519173,91.913587,27.307119,20.172737,35.939966,4.922048,2.615500,5.076339
min,9.700000e+04,-61.500000,-65.500000,1.000000,-85.700000,-92.900000,0.000000,-87.500000,-92.900000,0.000000,-87.500000,-94.700000,0.000000,-435.000000,-548.000000,0.000000,-23.100000,-42.500000
25%,4.600000e+05,-1.600000,0.000000,3.000000,-16.700000,-28.600000,3.000000,-18.200000,-28.600000,1.000000,-12.850000,-33.300000,11.000000,-2.000000,-10.000000,100.300000,-0.900000,-1.600000
50%,6.330000e+05,0.000000,10.400000,6.000000,0.000000,0.000000,6.000000,0.000000,0.000000,2.000000,0.000000,0.000000,17.000000,0.000000,0.000000,102.500000,0.000000,0.400000
75%,8.300000e+05,3.500000,21.300000,10.000000,20.000000,33.300000,11.000000,19.200000,25.000000,5.000000,7.000000,0.000000,32.000000,3.000000,7.000000,105.300000,0.900000,3.400000
max,2.400000e+06,169.600000,335.700000,10715.000000,700.000000,1400.000000,13145.000000,500.000000,1000.000000,28027.000000,900.000000,1500.000000,556.000000,515.000000,535.000000,137.600000,38.100000,61.200000


In [ ]:
make_histogram(df['Median Sale Price'], 50)

In [ ]:
sum(pd.isnull(df['New Listings YoY'])) # how to deal with missing values?

In [ ]:
df = df.fillna(0)

In [ ]:
df.describe()